#### Create embeddings and obtain items from menu

Setting up the environment

In [1]:
# Below libraries should be installed (Python=3.10):

# pip install langchain==0.0.209
# pip install nltk==3.8.1
# pip install tiktoken==0.4.0
# pip install openai==0.27.8
# pip install qdrant-client==1.2.0 #VectorStore
# pip install cohere==4.11.2 #For embeddings

#If a docsearch different than Qdrant is used, install the following:
# pip install faiss-cpu==1.7.4
# pip install chromadb==0.3.26
# pip install deeplake==3.6.6
# pip install pinecone-client==2.2.2

Import dependencies

In [2]:
from langchain import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory, VectorStoreRetrieverMemory
#Explanation of some common types of memory: https://github.com/pinecone-io/examples/blob/master/generation/langchain/handbook/03-langchain-conversational-memory.ipynb
#'ConversationSummaryBufferMemory' that summarizes chat also used but not helpful since small conversation and not saving any tokens. 
#Detailed list of memory options in https://api.python.langchain.com/en/latest/modules/memory.html#

from langchain.chains import ConversationalRetrievalChain, ConversationChain, RetrievalQA
from langchain.chains.question_answering import load_qa_chain
#Explanation of each QA option: https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

from langchain.embeddings import CohereEmbeddings
# from langchain.embeddings.openai import OpenAIEmbeddings

#Detailed list in https://api.python.langchain.com/en/latest/modules/embeddings.html
#For HuggingFace Embeddings https://medium.com/@ryanntk/choosing-the-right-embedding-model-a-guide-for-llm-applications-7a60180d28e3

from langchain.vectorstores import Qdrant
# from langchain.vectorstores import FAISS
# from langchain.vectorstores import Chroma
# from langchain.vectorstores import DeepLake

#More details on how to use Qdrant on https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/qdrant
#Detailed list in https://api.python.langchain.com/en/latest/modules/vectorstores.html. Pinecone (save in cloud) tried but gave errors
#Some other suggestions are Postgres with pgvector extension or supabase. None of them works (require extra parameters).
#Suggestions taken from https://www.reddit.com/r/LangChain/comments/12ia7nc/what_is_the_best_vectorstore_to_selfhost_your/

import os
import tiktoken
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file with OpenAI key

#Set API key for OpenAI
openai.api_key = openai_api_key=os.environ['OPENAI_API_KEY']

# from langchain.document_loaders import DirectoryLoader #Did not work

Create the menu by combining all txt files in one list. It does not worth translating to English first since embeddings search will not work better with it.

In [3]:
documents=[]
for file in os.listdir('./final_best_with_GPT/'):
    if 'final_' in file and 'final_website_preprocessed' not in file:
        with open('./final_best_with_GPT/' + file, 'r') as f:
            documents.append(f.read())
documents

['amstel 330ml - €2.00\namstel 500ml - €2.40\ncoca cola 1500ml - €3.00\ncoca cola 330ml - €1.60\ncoca cola 500ml - €1.80\ncoca cola light 330ml - €1.60\ncoca cola zero 1500ml - €3.00\ncoca cola zero 330ml - €1.60\ncoca cola zero 500ml - €1.80\nfanta λεμονίτα 330ml - €1.60\nfanta πορτοκαλάδα 1500ml - €3.00\nfanta πορτοκαλάδα με ανθρακικό 330ml - €1.60',
 'halloumi burger (μοσχαρίσιο, χαλούμι, ρόκα, ντομάτα, μαγιονέζα) - €5.30\nnumber 1 burger (μοσχαρίσιο, διπλό cheddar, αυγό τηγανητό, μαρούλι, ντομάτα, καραμελωμένα κρεμμύδια) - €5.50\nshrimp burger (παναρισμένη γαρίδα, ντομάτα, κρεμμύδι, μαρούλι, αβοκάντο, σως cocktail) - €5.20\nφτιάξε το δικό σου burger! - €0.80\nhamburger (μοσχαρίσιο, ντομάτα, κετσαπ, αγγουρομαγιονέζα) - €3.60\ncheese burger (μοσχαρίσιο, τυρί, ντομάτα, κετσαπ, αγγουρομαγιονέζα) - €3.90\nsuper burger (μοσχαρίσιο, ντομάτα, μπέικον, παρμεζάνα, κετσαπ, αγγουρομαγιονέζα) - €4.00\nmega burger (διπλό μοσχαρίσιο, διπλό τυρί, μπέικον, μαρούλι, κρεμμύδι, ντομάτα, κετσαπ, μαγιον

Make each element in the list being an item from the menu - Does not work well with embeddings, needs more context

In [4]:
# documents_final=[]
# longest_text=0
# for text in documents:
#     text = text.split('\n')
#     for item in text:
#         documents_final.append(item) #Get each item of menu as an element of list
#         if len(item)>longest_text:
#             longest_text=len(item) #Will be ~175 characters

# documents=documents_final #Replace our documents with the list of menu items

Make some changes in the menu

In [5]:
for ind, doc in enumerate(documents):
    if 'Κλασσική' or 'κλασσική' in doc:
        documents[ind] = doc.replace("Κλασσική", "πίτα" )
        documents[ind] = doc.replace("κλασσική", "πίτα" )

Count number of tokens before sending to OpenAI (text-embedding-ada-002-v2 (price 0.0004$/1K tokens)) and calculate price of embedding.
- Source: https://www.datasciencebyexample.com/2023/04/10/count-tokens-in-gpt-models-accurately/

An alternative way is to use the 'get_num_tokens' function from langchain OpenAI
- Source: https://api.python.langchain.com/en/latest/modules/llms.html#langchain.llms.OpenAI.get_num_tokens

Detailed list of pricing here: https://openai.com/pricing#language-models

In [7]:
all_docs="".join(documents) #Combine all texts to a string (needed below)

encoding = tiktoken.encoding_for_model("text-embedding-ada-002") #Returns the encoding

# calculuate the number of tokens 
num_token = len(encoding.encode(all_docs))

print("Number of tokens: ", num_token)
print("Total price: ", round(num_token*0.0004/1000,2),"$")

Number of tokens:  26406
Total price:  0.01 $


In [8]:
llm=OpenAI(openai_api_key=openai_api_key,temperature=0)
num_token=llm.get_num_tokens(all_docs)
print("Number of tokens (OpenAI function): ", num_token)

Number of tokens (OpenAI function):  31404


In [9]:
print("In comparison, total length of texts: ", len(all_docs))

In comparison, total length of texts:  32267


Create embeddings of the menu to be processed faster and cheaper

In [ ]:
# Get your text splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) #chunk_size refers to characters, not tokens!
# Split your documents into texts
texts = text_splitter.create_documents(documents)

#Turn your texts into embeddings
# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key,model='text-embedding-ada-002') #Alternative option (cheap but not free)
embeddings = CohereEmbeddings(model = "multilingual-22-12", cohere_api_key="meYcxVoNmXkYsapP5mgAWMYwJ3VMuulidqzmgaE1")
#Taken from #https://txt.cohere.com/search-cohere-langchain/
#Cohere Free tier limits: https://docs.cohere.com/docs/going-live
#Information about OpenAI embeddings: https://platform.openai.com/docs/guides/embeddings

#Get your docsearch ready
docsearch = Qdrant.from_documents(texts, embeddings,location=":memory:",  collection_name="my_documents", distance_func="Dot") # Local mode with in-memory storage only
# docsearch = FAISS.from_documents(texts, embeddings) #Performance similar to 'Qdrant'
# docsearch = Chroma.from_documents(texts, embeddings) #Seems worse than FAISS/Qdrant
# docsearch = DeepLake.from_documents(texts, embeddings) #Seems worse than FAISS/Qdrant
# For very long documents embeddings may not work well: https://github.com/hwchase17/langchain/issues/2442

#If we want to save them locally and load them for retrieval (Not used since quite fast for such small documents)
# docsearch.save_local("faiss_midjourney_docs")
# retriever=FAISS.load_local("faiss_midjourney_docs", OpenAIEmbeddings()).as_retriever(search_type="similarity", search_kwargs={"k":1})

Perform similarity search - Response time range 1 to 1.5 seconds

In [ ]:
query = "Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως."
docs = docsearch.similarity_search(query) #If OpenAI text-embedding-ada-002-v2 is used (price 0.0004/1K tokens), embedding for ~27K tokens around ~0.01$.
docs #set k=n to only get n most similar docs
#If 'similarity_search_with_score' used then we get "Attribute error tuple has no attribute 'page_content'".
#This can be fixed by setting docs = [item[0] for item in docs_and_scores] as suggested in https://github.com/hwchase17/langchain/issues/3790

[Document(page_content='πίτα με γυρο χοιρινό - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80\nπίτα με κεμπάπ - πατάτα, ντομάτα, κρεμμύδι, σως γιαουρτιού - €3.80\nπίτα με λουκάνικο φρανκφούρτης - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80\nπίτα με λουκάνικο χωριάτικο - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80\nπίτα με μπιφτέκι λαχανικών - πατάτα, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nπίτα με σνίτσελ κοτόπουλο - πατάτα, ντομάτα, σως κίτρινη - €3.80\nπίτα με σουβλάκι χοιρινό - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80\nπίτα με γύρο κοτόπουλο - πατάτα, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nπίτα με καλαμάρι - πατάτες, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nπίτα με κολοκυθοκεφτέδες - πατάτες, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nπίτα με ντοματοκεφτέδες - ντομάτα, μαρούλι, κέτσαπ - €3.80\nπίτα με σουβλάκι κοτόπουλο - πατάτα, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nκυπριακή με γύρο κοτόπουλο - €4.30\nκυπριακή με γυρο χοιρινό - €4.30', metadata={}),
 Document(page_con

In [ ]:
query = "3 σουβλάκι κοτόπουλο"
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='σουβλάκι κεμπάπ 100 γραμ. - €2.00\nσουβλάκι κοτόπουλο χειροποίητο 110 γραμ. - €2.00\nσουβλάκι χαλούμι - €1.80\nσουβλάκι χοιρινό χειροποίητο 100 γραμ. - €2.00\nσουβλάκι χωριάτικο λουκάνικο - €2.00', metadata={}),
 Document(page_content='πίτα με σουβλάκι κοτόπουλο - πατάτα, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nκυπριακή με γύρο κοτόπουλο - €4.30\nκυπριακή με γυρο χοιρινό - €4.30\nκυπριακή με καλαμάρι πατάτες, ντομάτα, κρεμμύδι, σως μουστάρδας - €4.30\nκυπριακή με κεμπάπ - €4.30\nκυπριακή με κολοκυθοκεφτέδες πατάτες, ντομάτα, κρεμμύδι, σως μουστάρδας - €4.30\nκυπριακή με λουκάνικο φρανκφούρτης - €4.30\nκυπριακή με λουκάνικο χωριάτικο - €4.30\nκυπριακή με μπιφτέκι λαχανικών - €4.30\nκυπριακή με ντοματοκεφτέδες μαρούλι, ντομάτα, κέτσαπ - €4.30\nκυπριακή με σνίτσελ κοτόπουλο - €4.30\nκυπριακή με σουβλάκι κοτόπουλο - €4.30\nκυπριακή με σουβλάκι χοιρινό - €4.30\nτορτίγια με γαρίδες, μαρούλι, λάχανο, καρότο, sweet chilly - €4.80\nτορτίγια με γύρο κοτόπουλο - €4.80\n

In [ ]:
query = "2 χοιρινα και μια πατατες" #Does not work well since two items that are not in the same document
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='πατάτες - €3.60\nπατάτες με σως φέτας & μπεικον - €4.80\nπατάτες dipper αλάτι, ρίγανη - €4.00\nπατάτες με gouda - €4.30\nπατάτες με gouda & μπέικον - €4.70\nπατάτες με σως επιλογής - €4.20', metadata={}),
 Document(page_content='πίτα με γυρο χοιρινό - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80\nπίτα με κεμπάπ - πατάτα, ντομάτα, κρεμμύδι, σως γιαουρτιού - €3.80\nπίτα με λουκάνικο φρανκφούρτης - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80\nπίτα με λουκάνικο χωριάτικο - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80\nπίτα με μπιφτέκι λαχανικών - πατάτα, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nπίτα με σνίτσελ κοτόπουλο - πατάτα, ντομάτα, σως κίτρινη - €3.80\nπίτα με σουβλάκι χοιρινό - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80\nπίτα με γύρο κοτόπουλο - πατάτα, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nπίτα με καλαμάρι - πατάτες, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nπίτα με κολοκυθοκεφτέδες - πατάτες, ντομάτα, κρεμμύδι, σως μουστάρδας - €3.80\nπίτα με ντοματ

In [ ]:
query = "2 χοιρινα"
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='σουβλάκι κεμπάπ 100 γραμ. - €2.00\nσουβλάκι κοτόπουλο χειροποίητο 110 γραμ. - €2.00\nσουβλάκι χαλούμι - €1.80\nσουβλάκι χοιρινό χειροποίητο 100 γραμ. - €2.00\nσουβλάκι χωριάτικο λουκάνικο - €2.00', metadata={}),
 Document(page_content='cheesecake αγριοκέρασο - €2.90\nμους σοκολάτας - €2.90\nπολίτικο κανταΐφι - €2.90\nπροφιτερόλ - €2.90', metadata={}),
 Document(page_content='schnitzel xοιρινό - €7.30\nσκαλοπίνια piccata - €8.90\nχοιρινά μπριζολάκια με μανιτάρια & θυμάρι - €8.50\nμερίδα μπιφτέκι κοτόπουλο (σερβίρεται με σαλάτα ή ρύζι) - €8.20\nμερίδα μπιφτέκι κοτόπουλο σκέτη - €6.00\nμερίδα γύρος χοιρινός (ντομάτα, κρεμμύδι, πίτα, τζατζίκι, παπρίκα γλυκιά) - €9.20\nμερίδα μπιφτέκι μοσχαρίσιο ν1 σκέτη - €6.00\nμερίδα φιλέτο κοτόπουλο (σερβίρεται με πατάτες ή ρύζι, σως) - €8.30\nμερίδα φιλέτο κοτόπουλο σκέτη - €6.00\nμισή μερίδα γύρος χοιρινός (ντομάτα, κρεμμύδι, πίτα, τζατζίκι, παπρίκα γλυκιά) - €5.50\nμοσχαρίσια γάλακτος (σερβίρεται με πατάτες ή ρύζι ή σαλάτα) - 

In [ ]:
query = "μια αστακομακαροναδα" #Does not exist and conversational approach needed to inform user
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='cheesecake αγριοκέρασο - €2.90\nμους σοκολάτας - €2.90\nπολίτικο κανταΐφι - €2.90\nπροφιτερόλ - €2.90', metadata={}),
 Document(page_content='αγκινάρες αλα πολίτα - 5.10 ευρώ\nαρακάς κοκκινιστός με πατατούλες - 5.00 ευρώ\nγαλέος σκορδαλιά - 9.50 ευρώ\nγεμιστά με πατάτες - 5.80 ευρώ\nγίγαντες - 5.10 ευρώ\nιμάμ - 5.50 ευρώ\nκαλαμάρι κρασάτο - 8.20 ευρώ\nκαλαμάρι με σπανάκι - 8.20 ευρώ\nκεφτεδάκια κοτόπουλο με σως γιαουρτιού, φρέσκια ντοματούλα, πατάτες τηγανιτές και πιτα - 7.30 ευρώ\nκεφτεδάκια της γιαγιάς - 6.90 ευρώ\nκολοκυθάκια αυγολέμονο με ρύζι και κιμά - 5.80 ευρώ\nκοτόπουλο γλυκόξινο - 6.90 ευρώ', metadata={}),
 Document(page_content='μακαρονοσαλάτα με αρακάπεννες, αρακάς, καπνιστό χοιρινό, καλαμπόκι, καρότο, κρεμμύδι φρέσκο, βινεγκρέτ - 4.00€\nσαλάτα εποχής μαρούλι ‘h λαχανο‘h αναμεικτή - 2.60€\nmista μαρούλι iceberg, λαχανο, ραντίτσιο, καρότο, εσκαρόλ, αγγούρι, βινεγκρέτ - 2.90€\nταμπουλέ πλιγούρι, ντομάτα, κρεμμύδι ξερό, κρεμμυδάκι φρέσκο, αγγούρι, καρότ

In [ ]:
query = "1 πατατες με σως"
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='πατάτες - €3.60\nπατάτες με σως φέτας & μπεικον - €4.80\nπατάτες dipper αλάτι, ρίγανη - €4.00\nπατάτες με gouda - €4.30\nπατάτες με gouda & μπέικον - €4.70\nπατάτες με σως επιλογής - €4.20', metadata={}),
 Document(page_content='κεφτεδάκια 8τμχ πατάτες σως - €5.60\nspring rolls λαχανικών 5τμχ - €4.60\nτυροκροκέτες 10τμχ πατάτες σως - €5.60\nανθότυρο ψητό - €3.80\nγραβιέρα - €4.20\nκολοκυθοκεφτέδες 5τμχ - €4.30\nμερ. ψωμι - €1.00\nμερ. ψωμι – σερβίτσιο - €1.20\nμερίδα μπιφτέκι λαχανικών πατάτα, μαρούλι, σως μουστάρδας - €7.00\nμερίδα πατατοσαλάτα - €4.00\nμερίδα πουρές πατάτας - €4.00\nμερίδα ρύζι basmati - €4.00', metadata={}),
 Document(page_content='cheesecake αγριοκέρασο - €2.90\nμους σοκολάτας - €2.90\nπολίτικο κανταΐφι - €2.90\nπροφιτερόλ - €2.90', metadata={}),
 Document(page_content='αγκινάρες αλα πολίτα - 5.10 ευρώ\nαρακάς κοκκινιστός με πατατούλες - 5.00 ευρώ\nγαλέος σκορδαλιά - 9.50 ευρώ\nγεμιστά με πατάτες - 5.80 ευρώ\nγίγαντες - 5.10 ευρώ\nιμάμ - 5.5

In [ ]:
query = "1 κοκα κολα" #Does not work well since description of that is in English in the menu
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='αγκινάρες αλα πολίτα - 5.10 ευρώ\nαρακάς κοκκινιστός με πατατούλες - 5.00 ευρώ\nγαλέος σκορδαλιά - 9.50 ευρώ\nγεμιστά με πατάτες - 5.80 ευρώ\nγίγαντες - 5.10 ευρώ\nιμάμ - 5.50 ευρώ\nκαλαμάρι κρασάτο - 8.20 ευρώ\nκαλαμάρι με σπανάκι - 8.20 ευρώ\nκεφτεδάκια κοτόπουλο με σως γιαουρτιού, φρέσκια ντοματούλα, πατάτες τηγανιτές και πιτα - 7.30 ευρώ\nκεφτεδάκια της γιαγιάς - 6.90 ευρώ\nκολοκυθάκια αυγολέμονο με ρύζι και κιμά - 5.80 ευρώ\nκοτόπουλο γλυκόξινο - 6.90 ευρώ', metadata={}),
 Document(page_content='σολομός με φαρφάλες (σολομός, κολοκυθάκι, ντοματίνια, ανήθο, βότκα, κρέμα γάλακτος) - €9.90\nσολομός ψητός (σολομός σχάρας. σερβίρεται με ψητά λαχανικά) - €12.70\nαγκινάρες αλα πολίτα - 5.10 ευρώ\nαρακάς κοκκινιστός με πατατούλες - 5.00 ευρώ\nγαλέος σκορδαλιά - 9.50 ευρώ\nγεμιστά με πατάτες - 5.80 ευρώ\nγίγαντες - 5.10 ευρώ\nιμάμ - 5.50 ευρώ\nκαλαμάρι κρασάτο - 8.20 ευρώ\nκαλαμάρι με σπανάκι - 8.20 ευρώ\nκεφτεδάκια κοτόπουλο με σως γιαουρτιού, φρέσκια ντοματούλα, πα

In [ ]:
query = "1 coca cola"
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='amstel 330ml - €2.00\namstel 500ml - €2.40\ncoca cola 1500ml - €3.00\ncoca cola 330ml - €1.60\ncoca cola 500ml - €1.80\ncoca cola light 330ml - €1.60\ncoca cola zero 1500ml - €3.00\ncoca cola zero 330ml - €1.60\ncoca cola zero 500ml - €1.80\nfanta λεμονίτα 330ml - €1.60\nfanta πορτοκαλάδα 1500ml - €3.00\nfanta πορτοκαλάδα με ανθρακικό 330ml - €1.60', metadata={}),
 Document(page_content='σως bbq - €1.50\nσως bbq μικρή - €0.70\npinza με αλλαντικά - €5.90\npinza με γύρο κοτόπουλο - €5.90\npinza με γύρο χοιρινό - €5.90\namstel 330ml - €2.00\namstel 500ml - €2.40\ncoca cola 1500ml - €3.00\ncoca cola 330ml - €1.60\ncoca cola 500ml - €1.80\ncoca cola light 330ml - €1.60\ncoca cola zero 1500ml - €3.00\ncoca cola zero 330ml - €1.60\ncoca cola zero 500ml - €1.80\nfanta λεμονίτα 330ml - €1.60\nfanta πορτοκαλάδα 1500ml - €3.00\nfanta πορτοκαλάδα με ανθρακικό 330ml - €1.60\ncheesecake αγριοκέρασο - €2.90\nμους σοκολάτας - €2.90\nπολίτικο κανταΐφι - €2.90\nπροφιτερόλ - €2.90

In [ ]:
query = "αφαιρεσε τις πατατες"
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='αγκινάρες αλα πολίτα - 5.10 ευρώ\nαρακάς κοκκινιστός με πατατούλες - 5.00 ευρώ\nγαλέος σκορδαλιά - 9.50 ευρώ\nγεμιστά με πατάτες - 5.80 ευρώ\nγίγαντες - 5.10 ευρώ\nιμάμ - 5.50 ευρώ\nκαλαμάρι κρασάτο - 8.20 ευρώ\nκαλαμάρι με σπανάκι - 8.20 ευρώ\nκεφτεδάκια κοτόπουλο με σως γιαουρτιού, φρέσκια ντοματούλα, πατάτες τηγανιτές και πιτα - 7.30 ευρώ\nκεφτεδάκια της γιαγιάς - 6.90 ευρώ\nκολοκυθάκια αυγολέμονο με ρύζι και κιμά - 5.80 ευρώ\nκοτόπουλο γλυκόξινο - 6.90 ευρώ', metadata={}),
 Document(page_content='κεφτεδάκια 8τμχ πατάτες σως - €5.60\nspring rolls λαχανικών 5τμχ - €4.60\nτυροκροκέτες 10τμχ πατάτες σως - €5.60\nανθότυρο ψητό - €3.80\nγραβιέρα - €4.20\nκολοκυθοκεφτέδες 5τμχ - €4.30\nμερ. ψωμι - €1.00\nμερ. ψωμι – σερβίτσιο - €1.20\nμερίδα μπιφτέκι λαχανικών πατάτα, μαρούλι, σως μουστάρδας - €7.00\nμερίδα πατατοσαλάτα - €4.00\nμερίδα πουρές πατάτας - €4.00\nμερίδα ρύζι basmati - €4.00', metadata={}),
 Document(page_content='πατάτες - €3.60\nπατάτες με σως φέτας &

In [ ]:
query = "τι εχω παραγγειλει μεχρι τωρα;" #Not be able to decide based on similarity search - Chat based approach needed for that
docs = docsearch.similarity_search(query) 
docs

[Document(page_content='hawaii μαρούλι iceberg, ραντίτσιο, κόκκινο λάχανο, κασιούς, ανανάς, κοτόπουλο, σουσάμι, βινεγκρέτ - 4.20€\nfussily a la tuna τόνο, κρεμμύδι φρέσκο & ξερό, ντομάτα καρέ, ανήθο, μαϊντανό, fussily, ελαιόλαδο - 4.20€\nmalibu μαρούλι iceberg, ραντίτσιο, κόκκινο λάχανο, πολυχρωμες πιπεριές, ρόκα, ντοματίνια, μανιτάρια, κασιούς, καρύδι, σταφίδες, βινεγκρέτ - 4.00€\nκεφτεδάκια 8τμχ πατάτες σως - €5.60\nspring rolls λαχανικών 5τμχ - €4.60\nτυροκροκέτες 10τμχ πατάτες σως - €5.60\nανθότυρο ψητό - €3.80\nγραβιέρα - €4.20\nκολοκυθοκεφτέδες 5τμχ - €4.30\nμερ. ψωμι - €1.00\nμερ. ψωμι – σερβίτσιο - €1.20\nμερίδα μπιφτέκι λαχανικών πατάτα, μαρούλι, σως μουστάρδας - €7.00\nμερίδα πατατοσαλάτα - €4.00\nμερίδα πουρές πατάτας - €4.00\nμερίδα ρύζι basmati - €4.00\nπατάτες - €3.60\nπατάτες με σως φέτας & μπεικον - €4.80\nπατάτες dipper αλάτι, ρίγανη - €4.00\nπατάτες με gouda - €4.30\nπατάτες με gouda & μπέικον - €4.70\nπατάτες με σως επιλογής - €4.20\nsweet chilly - €1.50\nαγγουρομαγι

Conversation with user by taking the whole or part (based on similarity) of menu as input - If the whole menu is used, it is too expensive, 0.06$ per message/call since used ~3K tokens with 0.02 per 1K (assumed that documents without an item per line used)

- Information on how to use 'load_qa_chain': https://python.langchain.com/docs/use_cases/question_answering.html

- Add memory to a chain with multiple inputs: https://python.langchain.com/docs/modules/memory/how_to/adding_memory_chain_multiple_inputs

- Templates, prompts, and memory should have specific keys based on which chain is used: https://github.com/hwchase17/langchain/issues/1800

In [ ]:
template = """You are a chatbot having a conversation with a human and taking an order from him. \
Given the menu below and a user input, create a final answer.
If the user input contains any items, search the menu for those items. If found, return the item and the price. \
If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. \

{context}

{chat_history}
Human: {human_input}
Chatbot:"""


prompt = PromptTemplate(input_variables=["chat_history", "human_input", "context"], template=template)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

#This method uses all text in menu
chain = load_qa_chain(OpenAI(openai_api_key=openai_api_key,temperature=0), chain_type="stuff", memory=memory, prompt=prompt)

In [ ]:
chain.memory.buffer #Check what is in memory

''

#### CAUTION! Just for the 10 prompts below (with k=4) cost would have been ~0.6$!

To reduce costs, we only give as input the most similar part of text and not the 4 most similar ones.

In practice the costs was 0.36$ for 8 prompts of ~18K tokens in total (only one with k=1). Maybe memorycache plays a role in that. For all 10 prompts below of ~9K tokens with k=1, price was 0.18$. There are slight variations of context length everytime we execute the prompts below.

In [ ]:
query = "Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως." #Sometimes adds extra materials based on similarity search
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) #Run the chain - Response time ranges from 3 to 7 seconds

' Η πίτα με γυρο χοιρινό έχει πατάτες, ντομάτα, κρεμμύδι και τζατζίκι και κοστίζει €3.80.'

In [ ]:
query = "3 σουβλάκι κοτόπουλο"
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Τα 3 σουβλάκια κοτόπουλο κοστίζουν €6.00.'

In [ ]:
query = "2 χοιρινα και μια πατατες" #Does not work well since two items that are not in the same document
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Τα 2 χοιρινά κοστίζουν €4.00 και η πατάτα €3.60.'

In [ ]:
query = "2 χοιρινα"
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Τα 2 χοιρινά κοστίζουν €4.00.'

In [ ]:
query = "μια αστακομακαροναδα" #Does not exist and conversational approach needed to inform user
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Δεν έχουμε αστακομακαρονάδα στο μενού.'

In [ ]:
query = "1 πατατες με σως"
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Η πατάτα με σως κοστίζει €4.80.'

In [ ]:
query = "1 κοκα κολα" #Does not work well since description of that is in English in the menu
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Δεν έχουμε κοκα κολα στο μενού.'

In [ ]:
query = "1 coca cola"
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Η Coca Cola 330ml κοστίζει €1.60.'

In [ ]:
query = "αφαιρεσε τις πατατες" #Cannot do that since just similarity check and not changes inputs
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Τα 2 χοιρινά κοστίζουν €4.00 και η Coca Cola 330ml κοστίζει €1.60.'

In [ ]:
#Probably will not run on top of all above since context length of 4097 tokens is exceeded
query = "τι εχω παραγγειλει μεχρι τωρα;" #Not be able to decide based on similarity search - Chat based approach needed for that
docs = docsearch.similarity_search(query,k=1)
chain.run(input_documents=docs, human_input=query, return_only_outputs=True) 

' Έχετε παραγγείλει 2 χοιρινά και μια Coca Cola 330ml.'

In [ ]:
print(chain.memory.buffer)

Human: Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως.
AI:  Η πίτα με γυρο χοιρινό έχει πατάτες, ντομάτα, κρεμμύδι και τζατζίκι και κοστίζει €3.80.
Human: 3 σουβλάκι κοτόπουλο
AI:  Τα 3 σουβλάκια κοτόπουλο κοστίζουν €6.00.
Human: 2 χοιρινα και μια πατατες
AI:  Τα 2 χοιρινά κοστίζουν €4.00 και η πατάτα €3.60.
Human: 2 χοιρινα
AI:  Τα 2 χοιρινά κοστίζουν €4.00.
Human: μια αστακομακαροναδα
AI:  Δεν έχουμε αστακομακαρονάδα στο μενού.
Human: 1 πατατες με σως
AI:  Η πατάτα με σως κοστίζει €4.80.
Human: 1 κοκα κολα
AI:  Δεν έχουμε κοκα κολα στο μενού.
Human: 1 coca cola
AI:  Η Coca Cola 330ml κοστίζει €1.60.
Human: αφαιρεσε τις πατατες
AI:  Τα 2 χοιρινά κοστίζουν €4.00 και η Coca Cola 330ml κοστίζει €1.60.
Human: τι εχω παραγγειλει μεχρι τωρα;
AI:  Έχετε παραγγείλει 2 χοιρινά και μια Coca Cola 330ml.


Another approach using RetrievalQA https://python.langchain.com/docs/modules/chains/popular/vector_db_qa

This method is the same as above. There would only be a difference if we haven't used the results of the similarity search above as input to the load_qa_chain.

For all 10 prompts (actually 9 requests) of ~22K tokens with k=1, price was 0.44$>0.18$ used above! Not sure exactly why this happens....

In [ ]:
# # Copy pasted from above - Replaced 'human_input' with 'question', otherwise it does not work

# template = """You are a chatbot having a conversation with a human and taking an order from him. \
# Given the menu below and a user input, create a final answer.
# If the user input contains any items, search the menu for those items. If found, return the item and the price. \
# If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. \

# {context}

# {chat_history}
# Human: {question}
# Chatbot:"""


# prompt = PromptTemplate(input_variables=["chat_history", "question", "context"], template=template)

# memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")

In [ ]:
# qa=RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=openai_api_key,temperature=0), #Retrieves and uses only relevant chunks from documents
#                                chain_type="stuff",
#                                retriever=docsearch.as_retriever(k=1),
#                                chain_type_kwargs={'prompt':prompt, "memory": memory}) #'verbose':True => in chain_type_kwargs
                               
# #Memory added within 'chain_type_kwargs': https://stackoverflow.com/questions/76240871/how-do-i-add-memory-to-retrievalqa-from-chain-type-or-how-do-i-add-a-custom-pr

Pass queries to RetrievalQA (from here on charges start)

In [ ]:
# query = "Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως." #Sometimes adds extra materials based on similarity search - Here also wrong price!
# qa.run(query)

' Η πίτα με γύρο πατάτες ντομάτα και σως κοστίζει €3.60.'

In [ ]:
# query = "3 σουβλάκι κοτόπουλο"
# qa.run(query)

' Τα 3 σουβλάκι κοτόπουλο κοστίζουν €6.00.'

In [ ]:
# query = "2 χοιρινα και μια πατατες" #Does not work well since two items that are not in the same document
# qa.run(query)

' Τα 2 χοιρινά κοστίζουν €4.80 και η πατάτα κοστίζει €3.60.'

In [ ]:
# query = "2 χοιρινα"
# qa.run(query)

' Τα 2 χοιρινά κοστίζουν €4.80.'

In [ ]:
# query = "μια αστακομακαροναδα" #Does not exist and conversational approach needed to inform user
# qa.run(query)

' Η αστακομακαρονάδα κοστίζει €2.90.'

In [ ]:
# query = "1 πατατες με σως"
# qa.run(query)

' Η πατάτα με σως κοστίζει €4.80.'

In [ ]:
# query = "1 κοκα κολα" #Does not work well since description of that is in English in the menu
# qa.run(query)

' Η κοκα κολα δεν είναι διαθέσιμη.'

In [ ]:
# query = "1 coca cola"
# qa.run(query)

' Η Coca Cola κοστίζει €1.60.'

In [ ]:
# query = "αφαιρεσε τις πατατες" #Cannot do that since just similarity check and not changes inputs
# qa.run(query)

' Η πατάτα με σως έχει αφαιρεθεί από την παραγγελία σας.'

In [ ]:
# #Probably will not run on top of all above since context length of 4097 tokens is exceeded
# query = "τι εχω παραγγειλει μεχρι τωρα;" #Not be able to decide based on similarity search - Chat based approach needed for that
# qa.run(query)

In [ ]:
# qa.combine_documents_chain.memory.chat_memory.messages #Show history of messages

[HumanMessage(content='Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως.', additional_kwargs={}, example=False),
 AIMessage(content=' Η πίτα με γύρο πατάτες ντομάτα και σως κοστίζει €3.60.', additional_kwargs={}, example=False),
 HumanMessage(content='3 σουβλάκι κοτόπουλο', additional_kwargs={}, example=False),
 AIMessage(content=' Τα 3 σουβλάκι κοτόπουλο κοστίζουν €6.00.', additional_kwargs={}, example=False),
 HumanMessage(content='2 χοιρινα και μια πατατες', additional_kwargs={}, example=False),
 AIMessage(content=' Τα 2 χοιρινά κοστίζουν €4.80 και η πατάτα κοστίζει €3.60.', additional_kwargs={}, example=False),
 HumanMessage(content='2 χοιρινα', additional_kwargs={}, example=False),
 AIMessage(content=' Τα 2 χοιρινά κοστίζουν €4.80.', additional_kwargs={}, example=False),
 HumanMessage(content='μια αστακομακαροναδα', additional_kwargs={}, example=False),
 AIMessage(content=' Η αστακομακαρονάδα κοστίζει €2.90.', additional_kwargs={}, example=False),
 HumanMessage(content='1 

Below is the only solution with a memory that can get a retriever as argument. If used with 'ConversationChain', the context length becomes too long and prices of items are made up. Approach above better and probably cheaper (here ~0.04$ per prompt).

In [ ]:
# #Copied from above - Removed 'context' and replaced 'chat_history' with 'history' and 'human_input' with 'input' - Otherwise it does not work
# template = """You are a chatbot having a conversation with a human and taking an order from him. \
# Given the menu below and a user input, create a final answer.
# If the user input contains any items, search the menu for those items. If found, return the item and the price. \
# If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. \

# {history}
# Human: {input}
# Chatbot:"""

# prompt = PromptTemplate(input_variables=["history", "input"], template=template)

In [ ]:
# retriever=docsearch.as_retriever()

In [ ]:
# #If below memory and chain are used, for the prompt and template we should copy the one from above but:
# # Remove 'context' and replace 'chat_history' with 'history' and 'human_input' with 'input' - Otherwise it does not work

# memory = VectorStoreRetrieverMemory(retriever=retriever)

# #Below gives the same results as if 'LLMChain' was used (if tested should be first imported).   
# conversation_with_summary = ConversationChain(
#    llm=OpenAI(openai_api_key=openai_api_key,temperature=0),
#    prompt=prompt,
#    memory=memory,
#    verbose=True)

In [ ]:
# conversation_with_summary.predict(input="Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως.")



> Entering new  chain...
Prompt after formatting:
You are a chatbot having a conversation with a human and taking an order from him. Given the menu below and a user input, create a final answer.
If the user input contains any items, search the menu for those items. If found, return the item and the price. If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. 
input: Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως.
response:  Η πίτα με γύρο πατάτες ντομάτα και σως κοστίζει €3.60.
input: Ωραία. Πρόσθεσε και 2 σουβλάκια κοτόπουλο και μια πατάτες με σως.
response:  Τα δύο σουβλάκια κοτόπουλο κοστίζουν €7.30 και η πατάτα με σως κοστίζει €3.60.
πίτα με γυρο χοιρινό - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80
πίτα με κεμπάπ - πατάτα, ντομάτα, κρεμμύδι, σως γιαουρτιού - €3.80
πίτα με λουκάνικο φρανκφούρτης - πατάτα, ντομάτα, κρεμμύδι, τζατζίκι - €3.80
πίτα με λουκάνικο χωριάτικ

' Η πίτα με γύρο πατάτες ντομάτα και σως κοστίζει €3.60.'

In [ ]:
# conversation_with_summary.predict(input="Ωραία. Πρόσθεσε και 2 σουβλάκια κοτόπουλο και μια πατάτες με σως.")
# #Should use a tool for counting. Even if new prompt clarifying that 2 souvlakia are added, it will not be able to count them.
# #If another prompt is added it will probably result in error due to context length of 4097 tokens exceeded



> Entering new  chain...
Prompt after formatting:
You are a chatbot having a conversation with a human and taking an order from him. Given the menu below and a user input, create a final answer.
If the user input contains any items, search the menu for those items. If found, return the item and the price. If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. 
πατάτες - €3.60
πατάτες με σως φέτας & μπεικον - €4.80
πατάτες dipper αλάτι, ρίγανη - €4.00
πατάτες με gouda - €4.30
πατάτες με gouda & μπέικον - €4.70
πατάτες με σως επιλογής - €4.20
input: Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως.
response:  Η πίτα με γύρο πατάτες ντομάτα και σως κοστίζει €3.60.
κεφτεδάκια 8τμχ πατάτες σως - €5.60
spring rolls λαχανικών 5τμχ - €4.60
τυροκροκέτες 10τμχ πατάτες σως - €5.60
ανθότυρο ψητό - €3.80
γραβιέρα - €4.20
κολοκυθοκεφτέδες 5τμχ - €4.30
μερ. ψωμι - €1.00
μερ. ψωμι – σερβίτ

' Τα δύο σουβλάκια κοτόπουλο κοστίζουν €7.30 και η πατάτα με σως κοστίζει €3.60.'

Using 'ConversationalRetrievalChain' to have a proper conversation. It is the same as RetrievalQA but also with access to chat history. 

- Results in made up prices and items
- Takes too long (8-13 seconds) since it feeds output prompt to another chain.
- Price for 3 prompts is ~0.07$

In [ ]:
# #Copied from above - Removed 'context' and replaced 'human_input' with 'input' - Otherwise it does not work
# template = """You are a chatbot having a conversation with a human and taking an order from him. \
# Given the menu below and a user input, create a final answer.
# If the user input contains any items, search the menu for those items. If found, return the item and the price. \
# If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. \

# {chat_history}
# Human: {input}
# Chatbot:"""

# prompt = PromptTemplate(input_variables=["chat_history", "input"], template=template)

In [ ]:
# retriever=docsearch.as_retriever()

In [ ]:
# memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# memory.chat_memory.add_user_message(template)

# qa=ConversationalRetrievalChain.from_llm(llm=OpenAI(openai_api_key=openai_api_key,temperature=0),retriever=retriever,memory=memory,verbose=True)

In [ ]:
# query = "Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως." #Wrong price!
# qa.run(question=query)



> Entering new  chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: You are a chatbot having a conversation with a human and taking an order from him. Given the menu below and a user input, create a final answer.
If the user input contains any items, search the menu for those items. If found, return the item and the price. If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. 
{chat_history}
Human: {input}
Chatbot:
Follow Up Input: Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως.
Standalone question:

> Finished chain.


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know

' Ναι, μπορούμε να σας προσφέρουμε μια πίτα με γύρο πατάτες ντομάτα και σως. Κοστίζει €3.60.'

In [ ]:
# query = "Ωραία. Πρόσθεσε και 2 σουβλάκια κοτόπουλο και μια πατάτες με σως. " #wrong price!
# qa.run(query)



> Entering new  chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: You are a chatbot having a conversation with a human and taking an order from him. Given the menu below and a user input, create a final answer.
If the user input contains any items, search the menu for those items. If found, return the item and the price. If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. 
{chat_history}
Human: {input}
Chatbot:
Human: Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως.
Assistant:  Ναι, μπορούμε να σας προσφέρουμε μια πίτα με γύρο πατάτες ντομάτα και σως. Κοστίζει €3.60.
Follow Up Input: Ωραία. Πρόσθεσε και 2 σουβλάκια κοτόπουλο και μια πατάτες με σως. 
Standalone question:

> Finished chain.


> Entering new  chain...


> 

' Τα δύο σουβλάκια κοτόπουλο κοστίζουν €7.30 και η πατάτα με σως κοστίζει €3.60.'

In [ ]:
# query = "Τέλος θα ήθελα μια αστακομακαρανόδα." #Made up answer!
# qa.run(query)



> Entering new  chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: You are a chatbot having a conversation with a human and taking an order from him. Given the menu below and a user input, create a final answer.
If the user input contains any items, search the menu for those items. If found, return the item and the price. If not found say that the item is not available. Respond in Greek. Do not make suggestions to the user. Do not make things up and only look the provided menu. 
{chat_history}
Human: {input}
Chatbot:
Human: Θα ήθελα να παραγγείλω μια πίτα γύρο πατάτες ντομάτα σως.
Assistant:  Ναι, μπορούμε να σας προσφέρουμε μια πίτα με γύρο πατάτες ντομάτα και σως. Κοστίζει €3.60.
Human: Ωραία. Πρόσθεσε και 2 σουβλάκια κοτόπουλο και μια πατάτες με σως. 
Assistant:  Τα δύο σουβλάκια κοτόπουλο κοστίζουν €7.30 και η πατάτα με σως κοστίζ

' Η αστακομακαρανόδα κοστίζει €10.30.'

#### Conclusions

- Best approach is the load_qa_chain but relatively expensive.
- Next best option is just similarity search and feed the result of the chat to the main conversation with user (where the actual order takes place with speech-to-text translation). In similarity search use k=1 since it's perform the best and is the cheapest option.

- A very useful post with explanations on what embeddings are, what are vector stores, how to use them within langchain, along with advantages of langchain (makes it easier and implementation in fewer lines of code) can be found in: https://medium.com/sopmac-ai/vector-databases-as-memory-for-your-ai-agents-986288530443